In [111]:
import matplotlib.pyplot as plt
import torch
from torch import nn


In [112]:
device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cuda'

In [113]:
weight = 0.5
bias = 1.8

x = torch.linspace(0, 6, 300).reshape(-1, 1)
x

tensor([[0.0000],
        [0.0201],
        [0.0401],
        [0.0602],
        [0.0803],
        [0.1003],
        [0.1204],
        [0.1405],
        [0.1605],
        [0.1806],
        [0.2007],
        [0.2207],
        [0.2408],
        [0.2609],
        [0.2809],
        [0.3010],
        [0.3211],
        [0.3411],
        [0.3612],
        [0.3813],
        [0.4013],
        [0.4214],
        [0.4415],
        [0.4615],
        [0.4816],
        [0.5017],
        [0.5217],
        [0.5418],
        [0.5619],
        [0.5819],
        [0.6020],
        [0.6221],
        [0.6421],
        [0.6622],
        [0.6823],
        [0.7023],
        [0.7224],
        [0.7425],
        [0.7625],
        [0.7826],
        [0.8027],
        [0.8227],
        [0.8428],
        [0.8629],
        [0.8829],
        [0.9030],
        [0.9231],
        [0.9431],
        [0.9632],
        [0.9833],
        [1.0033],
        [1.0234],
        [1.0435],
        [1.0635],
        [1.0836],
        [1

In [114]:
y_noiseless = weight * x + bias

noise_lev = 0.09
noise = torch.rand_like(y_noiseless) * noise_lev

y = y_noiseless + noise

indices = torch.randperm(x.size(0))

x = x[indices]
y = y[indices]

In [ ]:
test_split = int(0.2 * len(x))

x_train, y_train = x[test_split:], y[test_split:]
x_test, y_test = x[:test_split], y[:test_split]

In [ ]:
def plot_predictions(
        train_data=x_train,
        test_data=x_test,
        train_labels=y_train,
        test_labels=y_test,
        predictions=None
):
    plt.figure(figsize=(10, 10))
    plt.scatter(train_data, train_labels, s=4, c="b", label="training data")
    plt.scatter(test_data, test_labels, s=4, c="r", label="testing data")

    if predictions is not None:
        plt.scatter(test_data, predictions, s=4, c="g", label="predictions")

    plt.legend(prop={'size': 10})

In [ ]:
plot_predictions()

In [ ]:
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        # self.weight = nn.Parameter(torch.randn(1, dtype=torch.float32, requires_grad=True))
        # self.bias = nn.Parameter(torch.randn(1, dtype=torch.float32, requires_grad=True))
        self.linear = nn.Linear(1, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x)
        # return self.weight * x + self.bias

In [ ]:
model = LinearModel()

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
torch.manual_seed(42)

epochs = 500

epoch_counts = []
train_losses = []
test_losses = []

for epoch in range(epochs):
    model.train()

    train_pred = model(x_train)
    train_loss = loss_fn(train_pred, y_train)

    optimizer.zero_grad()
    train_loss.backward()

    optimizer.step()

    model.eval()

    with torch.inference_mode():
        test_pred = model(x_test)
        test_loss = loss_fn(test_pred, y_test)

        if epoch % 20 == 0:
            epoch_counts.append(epoch)
            train_losses.append(train_loss.detach().numpy())
            test_losses.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {train_loss} | MAE Test Loss: {test_loss} ")

In [ ]:
plt.plot(epoch_counts, train_losses, label="Train Loss")
plt.plot(epoch_counts, test_losses, label="Test Loss")
plt.title("Train vs Test loss graph")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
model.eval()
with torch.inference_mode():
    predictions = model(x_test)

plot_predictions(predictions=predictions)

In [132]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = "linear.pth"

MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(model.state_dict(), MODEL_SAVE_PATH)

In [133]:
linear_model = LinearModel()
linear_model.load_state_dict(torch.load(MODEL_SAVE_PATH))

<All keys matched successfully>

In [135]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.5192]], requires_grad=True),
 Parameter containing:
 tensor([1.7700], requires_grad=True)]